In [1]:
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats, signal, optimize
# from IntrinsicProfiles_Functions import Analyse_Neuron
pd.set_option('display.max_rows', 200, 'display.max_columns', 50)
np.set_printoptions(threshold=np.inf)


In [2]:
def loadLabBook(filename):
    data = pd.read_csv(filename)
    data["filename"] =  data["filename"].apply(lambda x: x if x.endswith(".abf") else x+".abf")
    data.to_csv(filename, index = False)
    return data

In [3]:
def LoadFileABF_CC(filename, dualPatch = 0):
    import pyabf
    abf = pyabf.ABF(filename)

    abf.setSweep(sweepNumber=0, channel=0)
    t = abf.sweepX
    sr = abf.dataRate
    V = np.empty((0,abf.sweepPointCount))
    I = np.empty((0,abf.sweepPointCount))
    
    firstChannel=0
    if dualPatch==2:
        firstChannel=2
        
    for sweep in range(abf.sweepCount):    
        abf.setSweep(sweepNumber=sweep, channel=firstChannel)
        V = np.vstack((V, abf.sweepY.reshape(1,-1)))
        abf.setSweep(sweepNumber=sweep, channel=firstChannel+1)
        I = np.vstack((I, abf.sweepY.reshape(1,-1)))
    
    ## Make output
    output = {
        "Voltage": V,
        "Current": I,
        "Time": t,
        "SamplingRate": sr
    } 
    
    return output

In [4]:
def Analyse_Tau(V, I, t, sr, plotting = False):
    #Start-end current command
    dI = np.diff(I, prepend = I[0])
    I_peaks,_ = signal.find_peaks(-dI,height=3)
    assert np.shape(I_peaks)[0]
    I_OFF = np.argmax(dI)
    I_ON = np.argmin(dI)

    # Decay time constant
    def mono_exp(x, a, b):
        return a * np.exp(b * x)

    xfit = t[I_ON:I_OFF]
#     yfit = V[I_ON:I_OFF]-np.mean(V[I_OFF-int(0.005*sr):I_OFF])
    yfit = V[I_ON:I_OFF]-np.min(V[I_ON:I_OFF])

    popt,pcov = optimize.curve_fit(mono_exp, xfit, yfit, method = 'lm', p0=(1.0,1.0), maxfev = 15000)
    Tau = 1/(-popt[1])     

    #Plot tau fitting
    if plotting:
        interpolX=t[I_ON-int(0.02*sr):I_OFF]
        plt.figure()
        plt.plot(xfit, yfit, 'k', label="Data")
        plt.plot(interpolX, mono_exp(interpolX, *popt), 'r-', label="Fitted Curve")    
        plt.ylim(yfit.min(),yfit.max())
        plt.legend()
        plt.show()

    return Tau

In [5]:
def IntrinsicProfile_AnalyseSweep(V, I, t, sr, plotting = False):
    # Initialise output variables
    commandDuration = 0
    I_command = 0
    V_Ih = np.nan
    V_SS = np.nan
    AP_number = np.nan
    AP_firingFrequency = np.nan
    AP_threshold = np.nan
    AP_spikeHeight = np.nan
    AP_halfWidth = np.nan
    AP_fAHP = np.nan
    AP_adaptationIndexLast = np.nan 
    AP_adaptationIndexSecond = np.nan
    Tau = np.nan
    Rin = np.nan
    RMP = np.median(V[0:int(0.01*sr)])    

    # Current command
    dI = np.diff(I, prepend=[I[0]])
    I_peaks,_ = signal.find_peaks(dI,height=3)

    if np.shape(I_peaks)[0]>0: ## All but zero current sweep
        I_ON = np.argmax(dI)
        I_OFF = np.argmin(dI)
        if I_ON > I_OFF: # Negative current steps
            I_OFF = np.argmax(dI)
            I_ON = np.argmin(dI)

        commandDuration=t[I_OFF]-t[I_ON]
        I_command=np.round(np.mean(I[I_ON+int(0.1*sr):I_ON+int(0.2*sr)])-np.mean(I[I_ON-int(0.2*sr):I_ON-int(0.1*sr)]))
        
        # Resting membrane potential
        RMP = np.median(V[I_ON-int(0.01*sr):I_ON])    
        
        if I_command < 0: #Hyperpolarizing current steps
            # Hyperpolarization-induced voltage sag
            V_Inst_idx = np.argmin(V[I_ON:I_OFF])+I_ON
            V_Inst = V[V_Inst_idx]
            V_SS = np.median(V[I_OFF-int(0.01*sr):I_OFF])
            V_Ih = V_SS - V_Inst
            
            Rin = ((RMP-V_SS)/abs(I_command))*1000 # MOhm
            
            if (I_command >= -45) & (I_command <= -10):
                Tau = Analyse_Tau(V, I, t, sr, plotting)                
                if (Tau < 0) | (Tau > 0.2):
                    Tau = np.nan

        else: # Depolarising current steps
            #Firing frequency: number action potentials (APs)
            V_peaks, _ = signal.find_peaks(V[I_ON:I_OFF+int(.002*sr)], height = -10, prominence = 20, distance = int(.0005*sr))
            AP_number = np.shape(V_peaks)[0]
            AP_firingFrequency = AP_number/commandDuration

            #Find firing frequency adaptation
            if np.shape(V_peaks)[0] >= 3:

                #Simon's method
                adaptationWindow = 0.1
                spikeN_i = np.count_nonzero((V_peaks) < adaptationWindow*sr)
                spikeN_f = np.count_nonzero(((V_peaks) > (commandDuration-adaptationWindow)*sr) & ((V_peaks) < commandDuration*sr))
                if spikeN_i != 0:
                    AP_adaptationIndexLast = (spikeN_i-spikeN_f)/spikeN_i*100

                # Scala et al., 2020 method: ratio between second inter-spike interval and first interspike interval
                AP_adaptationIndexSecond = (V_peaks[2]-V_peaks[1])/(V_peaks[1]-V_peaks[0])

            # Action potential features
            if np.shape(V_peaks)[0] in [1,2,3]:
                V_peaks = V_peaks + I_ON
                if np.shape(V_peaks)[0] in [2,3]:
                    end_idx = (V_peaks)[1]
                else:
                    end_idx = I_OFF
                
                if V_peaks[0] < int(.45*sr)+I_ON: #Spike has to start in the first 450 ms from I_ON
                    AP_window_start = int(0.005*sr)  # Before AP peak
                    AP_window = np.arange(V_peaks[0]-AP_window_start,end_idx)
                    AP_threshold, AP_spikeHeight, AP_halfWidth, AP_fAHP = IntrinsicProfile_Analyse_Spike(V[AP_window], t[AP_window], sr, AP_window_start, plotting)  


    return [commandDuration, I_command, RMP, V_SS, V_Ih, AP_number, AP_firingFrequency, AP_threshold, AP_spikeHeight, AP_halfWidth, AP_fAHP, AP_adaptationIndexLast, AP_adaptationIndexSecond, Tau, Rin]

In [6]:
def IntrinsicProfile_Analyse_Spike(V, t, sr, spikeIdx, plotting = False):
    
    # AP threshold: first derivative method, 10mV/ms cut-off
    thresholdCutOff = 10 #mV/ms
    dV = np.diff(V,n=1,prepend=[V[0]])
    dV=dV/((1/sr)*10**3) # Convert to mv/ms
    dV_peaks, _ = signal.find_peaks(dV,height=20)
    
    if dV_peaks.shape[0]==0: # Spike found if its derivative peak > 20 mV/ms, otherwise return NaNs
        AP_threshold = np.nan
        AP_spikeHeight = np.nan 
        AP_halfWidth = np.nan
        AP_fAHP = np.nan
    
    else:
        #  AP threshold
        dV_peaks = dV_peaks[0] #Select first spike    
        Th_idx = np.argmin(np.array(abs(dV[:dV_peaks]-thresholdCutOff)))
        AP_threshold = V[Th_idx]

        #  AP height
        AP_peakHeight = V[spikeIdx]
        AP_spikeHeight = AP_peakHeight-AP_threshold

        # AP half-width
        AP_halfHeigth = AP_spikeHeight/2
        endIdx = spikeIdx+int(0.02*sr)
        Half_Start = np.argmin(np.array(abs(V[Th_idx:spikeIdx]-(AP_peakHeight-AP_halfHeigth)))) + Th_idx
        Half_End = np.argmin(np.array(abs(V[spikeIdx:endIdx]-(AP_peakHeight-AP_halfHeigth)))) + spikeIdx
        AP_halfWidth = t[Half_End]-t[Half_Start]

        # Fast AHP (fAHP) amplitude
        fAHP_idx = np.argmin(V[spikeIdx:])+spikeIdx
        AP_fAHP = AP_threshold - V[fAHP_idx]

        #Plot AP and points of interest
        if plotting:
            plt.figure()
            plt.plot(t,V,'k')
            plt.plot(t[spikeIdx],V[spikeIdx],'rx')
            plt.plot(t[Th_idx],V[Th_idx],'rx')
            plt.plot(t[Half_Start],V[Half_Start],'bx')
            plt.plot(t[Half_End],V[Half_End],'bx')
            plt.plot(t[fAHP_idx],V[fAHP_idx],'bx')
            plt.xlim([t[spikeIdx]-0.005,t[spikeIdx]+0.05])

    return(AP_threshold, AP_spikeHeight, AP_halfWidth, AP_fAHP)

In [7]:
def IntrinsicProfiles_ExtractFeatures(data, plotting = False):
    # Obtain data
    V = data["Voltage"]
    I = data["Current"]
    t = data["Time"]
    sr = data["SamplingRate"]
    
    # Analyse sweeps 
    cols=['CurrentDuration','CurrentAmplitude','RMP','VoltageSteadyState','VoltageSag','ActionPotentialNumber','FiringFrequency','SpikeThreshold','SpikeHeight','SpikeHalfWidth','SpikeAHP','AdaptationIndexLast','AdaptationIndexSecond','Tau','Rin']
    tmpResults=pd.DataFrame(columns=cols)
    for sweep in range(V.shape[0]):
        tmpResults.loc[len(tmpResults), :] = IntrinsicProfile_AnalyseSweep(V[sweep,:], I[sweep,:], t, sr, plotting)
    tmpResults[cols] = tmpResults[cols].astype(float)  
    
    return tmpResults.to_dict(orient='list')

In [8]:
def IntrinsicProfiles_SelectFeatures(cellTable, IPdata, IPresults, plotting = False, plottingFolder = np.nan):  
    np.warnings.filterwarnings('ignore')

    #Input
    V = IPdata["Voltage"]
    I = IPdata["Current"]
    t = IPdata["Time"]
    sr = IPdata["SamplingRate"]
    
    CurrentDuration = np.array(IPresults["CurrentDuration"])
    CurrentAmplitude = np.array(IPresults["CurrentAmplitude"])
    RMP = np.array(IPresults["RMP"])
    VoltageSteadyState = np.array(IPresults["VoltageSteadyState"])
    VoltageSag = np.array(IPresults["VoltageSag"])
    ActionPotentialNumber = np.array(IPresults["ActionPotentialNumber"])
    FiringFrequency = np.array(IPresults["FiringFrequency"])
    SpikeThreshold = np.array(IPresults["SpikeThreshold"])
    SpikeHeight = np.array(IPresults["SpikeHeight"])
    SpikeHalfWidth = np.array(IPresults["SpikeHalfWidth"])
    SpikeAHP = np.array(IPresults["SpikeAHP"])
    AdaptationIndexLast = np.array(IPresults["AdaptationIndexLast"])
    AdaptationIndexSecond = np.array(IPresults["AdaptationIndexSecond"])
    Tau = np.array(IPresults["Tau"])
    Rin = np.array(IPresults["Rin"])
    RMP = np.array(IPresults["RMP"])
    
    # Firing
    MaxFiringFrequency = np.nanmax(FiringFrequency)
    
    if any(ActionPotentialNumber>10):
        AdaptationIndex = AdaptationIndexSecond[np.nanargmin(np.abs(ActionPotentialNumber-np.nanmax(ActionPotentialNumber)/2))]
    else:
        AdaptationIndex = np.median(np.array(AdaptationIndexSecond[ActionPotentialNumber>=3])[:5])
    
    # RMP and Rin
    xRin = CurrentAmplitude[(CurrentAmplitude >= -200) & (CurrentAmplitude < 0)]
    yRin = VoltageSteadyState[(CurrentAmplitude >= -200) & (CurrentAmplitude < 0)]

    slopeRin, interceptRin = np.polyfit(xRin, yRin, 1)
    xRin_fit = np.arange(-150, 50, 10)
    yRin_fit = slopeRin*xRin_fit + interceptRin
    
    if interceptRin < -45:
        RMP = interceptRin # mV
    else:
        RMP=np.median(RMP[:10])
    
    if (slopeRin*1000) > 0:
        Rin = slopeRin*1000 # MOhm
    else:
        Rin = Rin[~np.isnan(Rin)]
        Rin = Rin[-1]
    
    # Rheobase
    xRheobase = np.array(CurrentAmplitude[ActionPotentialNumber > 0])[:5]
    yRheobase = np.array(FiringFrequency[ActionPotentialNumber > 0])[:5]
    
    slopeRheobase, interceptRheobase = np.polyfit(xRheobase, yRheobase, 1)
    xRheobase_fit = np.arange(10, max(xRheobase), 10)
    yRheobase_fit = slopeRheobase*xRheobase_fit + interceptRheobase
    
    Rheobase = -interceptRheobase/slopeRheobase #pA   
    if (Rheobase < 5) | (Rheobase > 3000):
        Rheobase = xRheobase[0]
    
    # Voltage sag
    VoltageSag = VoltageSag[np.nanargmin(abs(VoltageSteadyState+100))]
    
    # Spike features
    AP_Threshold = SpikeThreshold[~np.isnan(SpikeThreshold)][0]
    AP_Height = SpikeHeight[~np.isnan(SpikeThreshold)][0]
    AP_HalfWidth = SpikeHalfWidth[~np.isnan(SpikeThreshold)][0]
    AP_AHP = SpikeAHP[~np.isnan(SpikeThreshold)][0]
    if AP_AHP < 0:
        AP_AHP=0
    
    #Tau
    Tau = Tau[~np.isnan(Tau)]
    if np.shape(Tau)[0]==0:
        Tau = np.nan
    else:
        Tau=Tau[-1]
    
#############################################################################################################################################################
    if plotting:
        #### Make figure
        assert type(plottingFolder)==str, "Error: Provide plotting folder"

        # Select sweeps to plot
        indexdV = np.nanargmin(np.array((abs(ActionPotentialNumber-10))))
        singleSpikeIdx = np.nonzero(ActionPotentialNumber>0)[0][0]
        maxSpikeIdx = np.nanargmax(FiringFrequency)
        sagIdx = np.nanargmin(VoltageSteadyState+100)

        fig = plt.figure(figsize=[20, 20])
        gs = gridspec.GridSpec(7, 2)
        plt.suptitle(cellTable.CellID.item() + ' - ' + cellTable.CellTarget.item() + '-' + cellTable.CellType.item() + ' - L' +str(int(cellTable.CellLayer.item())), fontsize=30)

        ax1 = plt.subplot(gs[0:3, 0])
        ax1.plot(t,V[maxSpikeIdx,:],'k')
        ax1.set_xlim((1, 1.8))
        ax1.set_ylim((-110, 30))
        plt.axis('off')

        ax2 = plt.subplot(gs[3:6, 0])
        ax2.plot(t,V[singleSpikeIdx,:],'k')
        ax2.plot(t,V[sagIdx,:],'k')
        ax2.plot((1.6, 1.8), (-100, -100),'k', linewidth=3)
        ax2.plot((1.6, 1.6), (-100, -70),'k', linewidth=3)
        plt.text(1.61, -87, '30 mV', fontsize=20)
        plt.text(1.64, -108, '200 ms', fontsize=20)

        ax2.set_xlim((1, 1.8))
        ax2.set_ylim((-110, 30))
        plt.axis('off')

        ax3 = plt.subplot(gs[6, 0])
        ax3.plot(t, I[singleSpikeIdx,:],'k')
        ax3.plot(t, I[sagIdx,:],'k')
        ax3.plot(t, I[maxSpikeIdx,:],'k')
        ax3.set_xlim((1, 1.8))
        ax3.plot((1.6, 1.6), (30, 50),'k', linewidth=3)
        plt.text(1.61, 34, '20 pA', fontsize=20)

        plt.axis('off')

        ax4 = plt.subplot(gs[0:4, 1])
        ax4.plot(CurrentAmplitude[CurrentAmplitude>=0], FiringFrequency[CurrentAmplitude>=0], 'ko')
        ax4.set_xlabel('Current (pA)', fontsize=20)
        ax4.set_ylim((0, np.nanmax(FiringFrequency)+5))
        ax4.set_ylabel('Firing Frequency (Hz)', fontsize=20)
        ax4.tick_params(axis='both', which='major', labelsize=20)
        ax4.spines['right'].set_visible(False)
        ax4.spines['top'].set_visible(False)
        ax4.spines['left'].set_linewidth(3)
        ax4.spines['bottom'].set_linewidth(3)
        ax4.xaxis.set_tick_params(width=3, length=10)
        ax4.yaxis.set_tick_params(width=3, length=10)
        plt.text(50,1,'RMP='+str(RMP.round())+' mV\nRin='+str(Rin.round())+' MOhm\nRheobase='+str(Rheobase.round())+' pA',fontsize = 20)
        
        ax5 = plt.subplot(gs[4:,1])
        ax5.plot(V[indexdV,:], np.diff(V[indexdV,:], prepend = V[indexdV,0] ), 'k-')
        ax5.set_ylabel('dV/dt', fontsize=20)
        ax5.set_xlabel('Voltage (mV)', fontsize=20)
        ax5.spines['left'].set_linewidth(3)
        ax5.spines['bottom'].set_linewidth(3)
        ax5.spines['top'].set_visible(False)
        ax5.spines['right'].set_visible(False)
        ax5.xaxis.set_tick_params(width=3, length=10)
        ax5.yaxis.set_tick_params(width=3, length=10)
        ax5.tick_params(axis='both', which='major', labelsize=20)
        ax5.tick_params(axis='y')


#         #### Rin and RMP plot
#         ax5 = plt.subplot(gs[4:,1])
#         ax5.plot(xRin, yRin, 'ro')
#         ax5.plot(xRin_fit, yRin_fit, 'k--')
#         ax5.plot([0, 0], [-500, 500], 'k', linewidth=0.5)
#         ax5.set_ylabel('Steady-state Vm (mV)', fontsize=20, color='r')
#         ax5.set_xlabel('Current (pA)', fontsize=20)
#         ax5.spines['left'].set_linewidth(3)
#         ax5.spines['bottom'].set_linewidth(3)
#         ax5.spines['top'].set_visible(False)
#         ax5.xaxis.set_tick_params(width=3, length=10)
#         ax5.yaxis.set_tick_params(width=3, length=10)
#         ax5.tick_params(axis='both', which='major', labelsize=20)
#         ax5.tick_params(axis='y', labelcolor='r')
#         ax5.set_ylim((yRin.min(), yRin.max()+10))

#         ax6 = ax5.twinx()  # instantiate a second axes that shares the same x-axis
#         ax6.plot(xRheobase, yRheobase, 'bo')
#         ax6.plot(xRheobase_fit, yRheobase_fit, 'k--')
#         ax6.plot([-500, 500], [0, 0], 'k', linewidth=0.5)
#         ax6.set_ylabel('Firing Frequency (Hz)', fontsize=20, color='b')
#         ax6.spines['right'].set_linewidth(3)
#         ax6.spines['top'].set_visible(False)
#         ax6.xaxis.set_tick_params(width=3, length=10)
#         ax6.yaxis.set_tick_params(width=3, length=10)
#         ax6.tick_params(axis='both', which='major', labelsize=20)
#         ax6.tick_params(axis='y', labelcolor='b')
#         ax6.set_xlim((-100, xRheobase.max()+5))
#         ax6.set_ylim((-1, yRheobase.max()+2))

        gs.update(hspace = 0.5, wspace =0.2)

        plt.savefig(os.path.join(outputFolder,cellTable.CellID.item()+'.pdf'), format='pdf', transparent=True)
        plt.savefig(os.path.join(outputFolder,cellTable.CellID.item()+'.tif'), format='tif', transparent=True)
        plt.close()
        
#############################################################################################################################################################
    if cellTable.MouseEyesOpen.item() == 1:
        MouseDevelopment = 'P14-P18'
    else:
        if cellTable.MouseAge.item() <= 8:
            MouseDevelopment = 'P5-P8'
        elif (cellTable.MouseAge.item() >= 9) & (cellTable.MouseAge.item() <= 13):
            MouseDevelopment = 'P9-P13'
    
    #Make output
    f = {
        "MouseID": cellTable.MouseID.item(),
        "MouseAge": cellTable.MouseAge.item(),
        "MouseDevelopment": MouseDevelopment,
        "BrainArea": cellTable.BrainArea.item(),
        "CellID": cellTable.CellID.item(),
        "CellTarget": cellTable.CellTarget.item(),
        "CellType": cellTable.CellType.item(),
        "CellLayer": cellTable.CellLayer.item(),
        "Rin": Rin,
        "RMP": RMP,
        "Tau": Tau,
        "Rheobase": Rheobase,
        "MaxFiringFrequency": MaxFiringFrequency,
        "AdaptationIndex": AdaptationIndex,
        "VoltageSag": VoltageSag,
        "AP_Threshold": AP_Threshold,
        "AP_Height": AP_Height,
        "AP_HalfWidth": AP_HalfWidth,
        "AP_AHP": AP_AHP
    } 
    
    np.warnings.filterwarnings('default')

    return f

In [9]:
# Inputs
filename = "C:\\Users\\Butt Lab\\OneDrive - OnTheHub - The University of Oxford\\TheBible.csv"
data = loadLabBook(filename)
data = data[data.Experiment=='IntrinsicProfile']
data = data[data.Discard==0]
results = pd.DataFrame()

# Select IDs to analyse
cellIDs = np.unique(data.CellID)
cellIDs = ["SA63.2"]

for cellID in cellIDs:  
    cellData = data[data.CellID==cellID]    
    print(cellID + ": Analysing... " + cellData.filename.values[0])
    inputFolder = os.path.join(list(cellData.foldername)[0], list(cellData.CellID)[0])
    outputFolder = os.path.join(inputFolder, "IntrinsicProfile")
    if not os.path.isdir(outputFolder):
        os.mkdir(outputFolder)
        
    abf = os.path.join(inputFolder, list(cellData.filename)[0])
    IPresultsFilename = os.path.join(outputFolder,"IP_Results.npy")
    
    IPdata = LoadFileABF_CC(abf, cellData.DualPatch.values[0])
    IPresults = IntrinsicProfiles_ExtractFeatures(IPdata, plotting = False)
    np.save(IPresultsFilename, IPresults)
    
    features = IntrinsicProfiles_SelectFeatures(cellData, IPdata, IPresults, plotting = False, plottingFolder = outputFolder)
    results = results.append(pd.DataFrame(features, index = [0]))
    
results.reset_index(inplace=True, drop=True)
results.to_csv('IntrinsicProfiles_Results.csv')
    

IndexError: index 0 is out of bounds for axis 0 with size 0